In [1]:
import os
from dotenv import load_dotenv

In [2]:
%cd ..
%pwd

e:\coding concept\Projects(Ai)\Medical_Chatbot


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'e:\\coding concept\\Projects(Ai)\\Medical_Chatbot'

In [3]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain.document_loaders import PyPDFLoader
file_path = "data/pdf_data.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()
data

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2020-09-15T15:08:34-04:00', 'author': 'Susana.Castillo', 'moddate': '2020-09-15T15:08:34-04:00', 'title': 'sdgs_targets_overview_resource.pdf', 'source': 'data/pdf_data.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion o

In [5]:
data_full = ""
for page in data:
    data_full+=page.page_content
data_full

'4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  Target 1.4 By 2030, ensure that all me

In [6]:
from langchain.text_splitter import TokenTextSplitter

splitter1 = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=10000,
    chunk_overlap=200
)

data_full_chunk = splitter1.split_text(data_full)

In [7]:
from langchain.docstore.document import Document
document_data = [Document(page_content=t) for t in data_full_chunk]
document_data

[Document(metadata={}, page_content='4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  Ta

In [8]:
splitter2 = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=1000,
    chunk_overlap=100
)

document_data2 = splitter2.split_documents(document_data)
document_data2

[Document(metadata={}, page_content='4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  Ta

In [9]:
from langchain_groq import ChatGroq

llm_ques_gen_pipeline = ChatGroq(
    model="openai/gpt-oss-120b",
    api_key=api_key
)

In [11]:
prompt_template = """
you are an expert at creating questions based on given material and context.
your goal is to prepare a student for their exams and test.
create subjective type questions only.
Instructions:
Generate only normal subjective questions.
Do not include target numbers, percentages, or technical formatting.
Questions should be open-ended, simple, and encourage descriptive answers.
create only questions do not add any any extra text remove all extra text
Keep them suitable for school/college exams or general assessments.
you do this by asking the questions about the text below:

...................
{text}
..................

create questions that will prepare the student for their tests. 
make sure not to lose any important information
"""

In [12]:
from langchain.prompts import PromptTemplate

PROMPT_QUESTIONS  = PromptTemplate(template=prompt_template, input_variables=["text"])

In [14]:
refine_template = ("""
You are an expert at creating practice subjective type questions based on given material and context.
Your goal is to help a student prepare for a his test.
Instructions:
Generate only normal subjective questions.
Do not include target numbers, percentages, or technical formatting.
create only questions do not add any any extra text remove all extra text
Questions should be open-ended, simple, and encourage descriptive answers.
Keep them suitable for school/college exams or general assessments.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [15]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [16]:
from langchain.chains.summarize import load_summarize_chain

ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [17]:
ques = ques_gen_chain.run(document_data)

print(ques)

C:\Users\HP\AppData\Local\Temp\ipykernel_11512\1145447928.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run(document_data)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

you are an expert at creating questions based on given material and context.
your goal is to prepare a student for their exams and test.
create subjective type questions only.
Instructions:
Generate only normal subjective questions.
Do not include target numbers, percentages, or technical formatting.
Questions should be open-ended, simple, and encourage descriptive answers.
create only questions do not add any any extra text remove all extra text
Keep them suitable for school/college exams or general assessments.
you do this by asking the questions about the text below:

...................
4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets

In [18]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()
vector_db = FAISS.from_documents(document_data2, embeddings)

C:\Users\HP\AppData\Local\Temp\ipykernel_11512\1371015979.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [19]:
llm_ans_gen_pipeline = ChatGroq(
    model="openai/gpt-oss-120b",
    api_key=api_key
)

In [20]:
from langchain.chains import RetrievalQA

answer_generation_chain = RetrievalQA.from_chain_type(
    llm=llm_ans_gen_pipeline,
    chain_type="stuff",
    retriever=vector_db.as_retriever()
)

In [25]:
ques_list = ques.split("\n")
new_ques_list = []
for i in range(0,20,2):
    new_ques_list.append(ques_list[i])
new_ques_list

['What are the primary aims of the first Sustainable Development Goal concerning poverty eradication?  ',
 'How can social protection systems be designed to support the poor and vulnerable populations?  ',
 'In what ways can equal rights to economic resources and basic services be ensured for all individuals?  ',
 'Describe strategies to build resilience of disadvantaged groups against climate‑related shocks and disasters.  ',
 'What types of resources and cooperation are needed to mobilize adequate support for ending poverty worldwide?  ',
 'Explain how policy frameworks can promote pro‑poor and gender‑sensitive development.  ',
 'What actions are necessary to achieve universal access to safe, nutritious and sufficient food?  ',
 'How can malnutrition in children, adolescents, pregnant women and the elderly be addressed?  ',
 'Identify measures that can double the productivity and incomes of small‑scale food producers.  ',
 'Discuss approaches to create sustainable food production sys

In [26]:
for question in new_ques_list:
    print("Question: ",question)
    answer = answer_generation_chain.run(question)
    print("Answer: ",answer)
    print("-----------------------------------")
    with open("answers.txt","a", encoding="utf-8") as f:
        f.write("Question: "+question+"\\n")
        f.write("Answer: "+answer+"\\n")
        f.write("--------------------------------------------------------")

Question:  What are the primary aims of the first Sustainable Development Goal concerning poverty eradication?  
Answer:  **Goal 1 – “End poverty in all its forms everywhere”** focuses on eliminating poverty and building the resilience of the poor and vulnerable. Its primary aims, as laid out in the SDG 1 targets, are to:

1. **Eradicate extreme poverty** – by 2030, end extreme poverty for all people (currently measured as living on less than $1.25 a day).  
2. **Halve overall poverty** – by 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions, according to national definitions.  
3. **Provide universal social protection** – implement nationally appropriate social‑protection systems and measures (including safety‑net floors) so that the poor and vulnerable have substantial coverage.  
4. **Guarantee equal economic rights and access** – ensure that all men and women—especially the poor and vulnerable—have equal right